In [1]:
import numpy as np
import osqp
import matplotlib.pyplot as plt
from scipy import sparse
import control as ct
from casadi import *
import dpilqr
import itertools

In [34]:
def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

In [35]:
from solvers.util import (
    compute_pairwise_distance,
    compute_pairwise_distance_nd_Sym,
    define_inter_graph_threshold,
    distance_to_goal,
    split_graph, 
    generate_f,
    generate_f_human_drone,
    objective,
    generate_min_max_input,
    generate_min_max_state
)


In [36]:
def linear_kinodynamics(dt,n_agent):
    #Decision vector is a = [a_x, a_y, a_z]
    #State vector is X = [p_x, p_y, p_z, v_x, v_y, v_z]
    #Discretization time step is dt
    A_tot = sparse.lil_matrix((6*n_agent, 6*n_agent))
    B_tot = sparse.lil_matrix((6*n_agent, 3*n_agent))
    A = sparse.csc_matrix([[1, 0, 0, dt, 0, 0],
                           [0, 1, 0, 0 , dt ,0],\
                           [0, 0, 1, 0, 0 , dt],\
                           [0, 0, 0, 1, 0 ,0],\
                           [0, 0, 0, 0, 1 ,0],\
                           [0, 0, 0, 0, 0, 1]])
    B = sparse.csc_matrix([[dt**2/2, 0, 0],\
                           [0, dt**2/2, 0],\
                           [0, 0, dt**2/2],\
                           [dt, 0, 0 ],\
                           [0, dt , 0],\
                           [0, 0, dt]])

    for i in range(n_agent):
        A_tot[i*6:(i+1)*6,i*6:(i+1)*6] = A
        B_tot[i*6:(i+1)*6,i*3:(i+1)*3] = B
        
    
    return A_tot, B_tot
        
        
    

In [37]:
A_tot,B_tot = linear_kinodynamics(0.1,3)

In [38]:
from solvers import util

In [39]:
x0,xr = util.paper_setup_3_quads()

In [40]:
def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))


In [41]:
def diag_block_mat_booliex(L):
    shp = L[0].shape
    mask = np.kron(np.eye(len(L)), np.ones(shp))==1
    out = np.zeros(np.asarray(shp)*len(L),dtype=int)
    out[mask] = np.concatenate(L).ravel()
    return out

In [42]:
def get_coll_cost(x, x_dims, radius, weight=200):
    distances = compute_pairwise_distance(x0, x_dims)
    pair_costs = np.fmin(np.zeros(1), distances - radius) ** 2
    return pair_costs.sum()*weight

In [43]:
def construct_P(P_base, num_agents):
    n = 6  # Dimension of state vector for each agent
    for i in range(num_agents):
        for j in range(num_agents):
            if i !=j:
                # Determine the position iices of agents i and j in the state vector
                pos_i = n * i  # Assuming each agent has 6 state vector components (3 positions + 3 velocities)
                pos_j = n * j

                # Modify the corresponding elements in P to incorporate collision avoidance
                # P_base[pos_i:pos_i + 3, pos_i:pos_i + 3] += np.eye(3) # (pos_i, pos_i)
                P_base[pos_i:pos_i + 3, pos_j:pos_j + 3] = np.eye(3)*50 # (pos_i, pos_j)
                P_base[pos_j:pos_j + 3, pos_i:pos_i + 3] = -np.eye(3)*50  # (pos_j, pos_i)
                # P_base[pos_j:pos_j + 3, pos_j:pos_j + 3] += np.eye(3) # (pos_j, pos_j)

    return P_base

## Solving the consensus ADMM problem with OSQP

In [44]:
import cvxpy as cp

In [45]:
def compute_pairwise_distance_CVX(X, x_dims, n_dims):
    """Analog to the above whenever some agents only use distance in the x-y plane"""
    CYLiER_RADIUS = 0.2

    n_states = x_dims[0]
    n_agents = len(x_dims)
    distances = []
    eps = 1e-3

    for i, n_dim_i in zip(range(n_agents), n_dims):
        for j, n_dim_j in zip(range(i + 1, n_agents), n_dims[i + 1 :]):
            n_dim = min(n_dim_i, n_dim_j)

            Xi = X[i * n_states : i * n_states + n_dim]
            Xj = X[j * n_states : j * n_states + n_dim]
            dX = Xi-Xj

            if n_dim == 3:
                # distances.append(cp.sqrt(dX[0]**2+dX[1]**2+dX[2]**2+eps))
                distances.append(cp.norm(dX))
            else:
                distances.append(cp.sqrt(dX[0]**2+dX[1]**2 + eps)+CYLiER_RADIUS)
    
    return distances

In [46]:
# # Solving MPC with OSQP solver
# N = 10

# cost = 0
# constr = []
# x0,xr = util.paper_setup_3_quads()


# nsim = 50
# radius = 0.5


# # X_full = np.zeros((nsim+1,nx))
# X_full = np.zeros((0,nx))
# X_full = np.r_[X_full,x0.reshape(1,-1)]
# xi = x0.flatten()
# xr = xr.flatten()
# Ad,Bd = linear_kinodynamics(0.1, 3)
# n_agents = 3
# nx = n_agents * 6
# nu = n_agents * 3
# Q = sparse.diags([10., 10., 10., 10., 10., 10.]*n_agents)
# QN = Q*10
# R = 0.1*sparse.eye(nu)
# s_prev = None

# iter = 0
# x_dims = [n_states]*n_agents
# n_dims = [3]*n_agents
# # while not np.all(dpilqr.distance_to_goal(xi,xr,3,6,3) < 0.1) and (iter <= nsim):
# for _ in range(nsim):
#     x = cp.Variable((N+1,nx))
#     u = cp.Variable((N,nu))
#     cost = 0
#     constr = []
    
#     for t in range(N):

#         cost += cp.quad_form(x[t+1,:],Q) + cp.quad_form(u[t,:],R)
  
#         # Linearized collision avoidance constraints
#         if iter > 0:
#             for i in range(n_agents):
#                 for j in range(n_agents):
#                     if j != i:
#                         for k in range(N+1):
#                             constr += [cp.norm(x_prev[k,j*n_states:j*n_states+3]-x_prev[k,i*n_states:i*n_states+3], 1) + \
#                                     (x_prev[k,j*n_states:j*n_states+3].T- x_prev[k,i*n_states:i*n_states+3].T)/cp.norm(x_prev[k,j*n_states:j*n_states+3]\
#                                     -x_prev[k,i*n_states:i*n_states+3], 1)@\
#                                     (x[k,j*n_states:j*n_states+3]-x[k,i*n_states:i*n_states+3]) >= radius]
    
                        
#         constr += [x[t + 1,:] == Ad @ x[t,:] + Bd @ u[t, :]]
#         constr += [u[t,:] <= np.tile(np.array([5, 5, 5]),(n_agents,)) ]
#         constr += [np.tile(np.array([-5, -5, -5]),(n_agents,)) <= u[t, :]]
#         # constr += [x[t, :] <= np.tile(np.array([np.inf, np.inf, np.inf, 2, 2, 2]),(n_agents,))]
#         # constr += [np.tile(-np.array([np.inf, np.inf, np.inf, 2, 2, 2]),(n_agents,)) <= x[t,:]]

#     cost += cp.quad_form((x[-1,:]-xr.flatten()),Q)    
#     constr += [x[0, :] == xi]

#     problem = cp.Problem(cp.Minimize(cost), constr)
#     problem.solve(verbose=True)

#     u_trj = u.value
#     x_trj = x.value
#     x_prev = x_trj
#     ctrl = u_trj[1]
#     xi = Ad@xi + Bd@ctrl
#     # X_full[iter+1,:] = xi
#     X_full = np.r_[X_full,xi.reshape(1,-1)]
    
#     iter +=1
    


In [47]:
# %matplotlib ipympl
# plt.figure(dpi=100)
# dpilqr.plot_solve(X_full,0,xr,[6,6,6],True,3)
# plt.legend(plt.gca().get_children()[1:3], ["Start Position", "Goal Position"])

### Consensus-ADMM

In [1]:
from solvers import util
from multiprocessing import Process, Pipe
import cvxpy as cp
from multi_agent_scp import *

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
# Number of terms f_i.
N = 3
Ad,Bd = linear_kinodynamics(0.1,N)
# A list of all the f_i.
T = 15 #Prediction horizon
n_states = 6
n_inputs = 3
Q_try = np.eye(n_states*N)
R_try = np.eye(n_inputs*N)*0.1
n = n_states*N

x0, xr = util.paper_setup_3_quads()

constr = []
cost_i = 0
f_list = []

nx = n_states*N
nu = n_inputs*N

# x = cp.Variable(((T+1)*nx, 1))
# u_input = cp.Variable((T*nu, 1))
y_state = cp.Variable(((T+1)*nx + T * nu, 1))

for id in range(N):
    objective = objective_admm(y_state, xr, T, Q_try, R_try, Q*10)
    f_list.append(objective)

def run_worker(f, pipe):
    Q = np.diag([10., 10., 10., 10., 10., 10.]* N)
    x_dims = [6]*3
    QN = Q*10
    P = 1e3*np.eye(nx) 
    R = 0.1*np.eye(nu)
    ρ = 200                        
    tol = 5e-1

    rho = 1
    xbar = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu))
    u = cp.Parameter((T+1)*nx + T*nu, value=np.zeros((T+1)*nx + T*nu)) #This is the scaled Lagrange multiplier
    f += (rho/2)*cp.sum_squares(y_state.flatten() - xbar + u)
    prox = cp.Problem(cp.Minimize(f),constr)
    # ADMM loop.
    iter = 0
    
    fd = jax.jit(discretize(multi_Quad_Dynamics, 0.1, x_dims))
    while True:
        try:
            #Solve local constrained problem with SCP 
            # s,u = solve_scp(fd, P, Q, R, T, xr, x0.reshape(1,-1), ρ, tol, 30, N, 0.5, f)
            
            prox.solve()
            
            # pipe.send(np.hstack((s.flatten(),u.flatten())))
            pipe.send(y_state.value.flatten())
            
            xbar.value = pipe.recv() #receive the averaged result from the main process.
            # print(f'xbar has shape {xbar.value.shape} \n')
            u.value += y_state.value.flatten() - xbar.value
            # u.value += np.hstack((s.flatten(),u.flatten())) - xbar.value
            iter += 1
            
        except EOFError:
            print("Connection closed.")
            break

# Setup the workers.
pipes = []
procs = []
for i in range(N):
    local, remote = Pipe()
    pipes += [local]
    procs += [Process(target=run_worker, args=(f_list[i], remote))]
    procs[-1].start()

# ADMM loop.
solution_list = []
for i in range(30):
    # Gather and average xi
    xbar = sum(pipe.recv() for pipe in pipes)/N
    # print(f'xbar is {xbar}, has shape {xbar.shape}\n')
    solution_list.append(xbar)
    # print(f'average of xbar is {np.mean(xbar)}\n')
    # Scatter xbar
    for pipe in pipes:
        pipe.send(xbar)
    
[p.terminate() for p in procs]

ValueError: dimension mismatch

In [158]:
xr.T

array([[2.5, 1.5, 1. , 0. , 0. , 0. , 0.5, 1.5, 1. , 0. , 0. , 0. , 1.5,
        2.2, 1. , 0. , 0. , 0. ]])

In [160]:
solution_list[-1][T*nx:(T+1)*nx] #state vector at the last step of the horizon T

array([ 0.173,  0.52 ,  0.347, -0.057, -0.17 , -0.113,  0.866,  0.52 ,
        0.347, -0.283, -0.17 , -0.113,  0.52 ,  0.451,  0.347, -0.17 ,
       -0.147, -0.113])